# Delta Method

Code for reproducing all the results for the **classical delta method** in the paper _The Delta-method and influence function in epidemiology: a reproducible tutorial_

In [ ]:
%pip install --upgrade --quiet --force-reinstall "sympy>=1.10.0"
%pip install --quiet "session_info"

     |████████████████████████████████| 6.4 MB 5.6 MB/s 
     |████████████████████████████████| 532 kB 41.7 MB/s 
     |████████████████████████████████| 63 kB 943 kB/s 


In [ ]:
#We'll use the sympy library for symbolic computation of variances and expected values
import sympy
from sympy import * 
from sympy.stats import Variance, Expectation, Covariance, Binomial
from sympy.stats.rv import RandomSymbol
import session_info
sympy.__version__   #version of sympy should be >=1.10

'1.10.1'

## Introduction to Sympy
[Sympy](https://docs.sympy.org/latest/guides/getting_started/install.html) is a computer algebra system for symbolic calculation. In order to use it you need to declare variables using `Symbol`:

In [ ]:
x = Symbol("x")
y = Symbol("y")

You can use the `Symbol`s to generate algebraic expressions such as:

In [ ]:
squarexy = pow((x + y),2) #(x + y)^2

which can be expanded, derived and integrated:

In [ ]:
expand(squarexy)

x**2 + 2*x*y + y**2

In [ ]:
diff(squarexy,x)

2*x + 2*y

In [ ]:
integrate(squarexy, x)

x**3/3 + x**2*y + x*y**2

Random variables can be declared with `RandomSymbol`:

In [ ]:
#Declaration of random variables can be made via RandomSymbol and their 
#variance and expected values can be estimated and simplified
X                = RandomSymbol('X')

#Properties of expectation can be computed
expectation_of_x = Expectation(3*X + 2)
expectation_of_x.expand()

2 + 3*Expectation(X)

In [ ]:
#As well as properties of variance
variance_of_x    = Variance(3*X + 2)
variance_of_x.expand()

9*Variance(X)

## 3.1 Delta-method for the mean (classical)

First we declare the constants: $\mu$, $\sigma^2$ and $n$ corresponding to the mean, variance and sample size. We also declare the random variable $\bar{X}$ corresponding to the sample mean which has mean $\mu$ and variance $\sigma^2/n$ 

In [ ]:
#Declaration of constants (Symbol) and random variables (RandomSymbol)
mu       = Symbol('mu')
sigmasq  = Symbol('sigma^2', positive = True) #Variance of x
n        = Symbol('n', positive = True, integer = True)
Xbar     = RandomSymbol('Xbar') 

We write the function $\phi$ as a function of $\mu$ _i.e._ $\phi(\mu) =  \mu$:

In [ ]:
#Declaring phi as a function of mu
def phi(mu):
    return mu

We calculate the derivative: $\frac{\partial \phi}{\partial \mu}$:

In [ ]:
#Obtaining the derivative
classical_derivative = derive_by_array(phi(mu), mu)

We then calculate the direction vector for the Hadamard derivative $v = \bar{X} - \mu$:

In [ ]:
v = Xbar - mu

The Hadamard derivative in the direction of $v$ (that is $ \partial_{\hat{\theta} - \theta}$) is thus:

In [ ]:
hadamard = classical_derivative*v

And the approximation is obtained by computing the right side of:
$$
\phi(\hat{\theta}) = \phi(\theta)  + \partial_{\hat{\theta} - \theta} \phi(\theta)
$$

In [ ]:
phi(mu) + hadamard

Xbar

Hence the Delta-method's approximation to the variance is:

In [ ]:
delta_variance = Variance(phi(mu) + hadamard).expand()

and we know from classical inference that $\text{Var}(\bar{X}) = \frac{\sigma^2}{n}$

In [ ]:
delta_variance.subs(Variance(Xbar), sigmasq/n)

sigma^2/n

## 3.3 Delta-method for the variance of the ratio of two sample means

Let $\{X_1, \dots, X_n\}$ denote a random sample of variables $X$ with mean $\mu_X$ and variance $\sigma^2_X$ and  $\{Y_1, \dots, Y_n\}$ denote a random sample of variables $Y$ with mean $\mu_Y$ and variance $\sigma^2_Y$ we are interested in approximating the variance of 

$$
\phi(\mu_X,\mu_Y) = \frac{\mu_X}{\mu_Y}
$$

to so we declare the symbols $\bar{X},\bar{Y},\mu_X,\mu_Y$ and the function $\phi$:

In [ ]:
#Declaration of variables
mu_x       = Symbol('mu_x')
mu_y       = Symbol('mu_y')
n          = Symbol('n', positive=True, integer = True) #Sample size of X and Y
sigmasq_x  = Symbol('sigma^2_X', positive = True) 
sigmasq_y  = Symbol('sigma^2_Y', positive = True) 
sigma_xy   = Symbol('sigma_xy') #Covariance
Xbar       = RandomSymbol('Xbar')
Ybar       = RandomSymbol('Ybar')  

#Declaration of function
def phi(mu_x, mu_y):
  return mu_x/mu_y

We calculate the derivative (in this case, gradient) in the direction of 
$$
v = \begin{pmatrix}
    \bar{X} - \mu_X\\
    \bar{Y} - \mu_Y
    \end{pmatrix}
$$

In [ ]:
#Obtain the direction vector
v = Matrix([Xbar - mu_x, Ybar - mu_y])

#And calculate the gradient
grad = derive_by_array(phi(mu_x,mu_y), [mu_x, mu_y])
grad = Matrix(grad)

#Thus calculating the Hadamard (directional) derivative:
hadamard = grad.dot(v)
hadamard

-mu_x*(-mu_y + Ybar)/mu_y**2 + (-mu_x + Xbar)/mu_y

The variance of the hadamard derivative (influence function) is estimated as follows:

In [ ]:
variance_expanded = Variance(hadamard).expand().simplify()
variance_expanded

(mu_x**2*Variance(Ybar) - 2*mu_x*mu_y*Covariance(Xbar, Ybar) + mu_y**2*Variance(Xbar))/mu_y**4

We can further simplify by substituting the variances of $\bar{X}$ and $\bar{Y} as well as the covariances$:

In [ ]:
variance_expanded = variance_expanded.subs(Covariance(Xbar,Ybar), sigma_xy/n)
variance_expanded = variance_expanded.subs(Variance(Xbar), sigmasq_x/n)
variance_expanded = variance_expanded.subs(Variance(Ybar), sigmasq_y/n)

variance_expanded.simplify()

(mu_x**2*sigma^2_Y - 2*mu_x*mu_y*sigma_xy + mu_y**2*sigma^2_X)/(mu_y**4*n)

## 3.4 Delta-method for the risk ratio
Consider the risk ratio for two probabilities defined by:
$$
RR(p_1,p_2) = \frac{p_1}{p_2}
$$
and the transformation $\phi$ given by:
$$
\phi(p_1,p_2) = \log\big(RR(p_1,p_2)\big)
$$
In this case, a random sample of $N_1$ elements of one category and $N_2$ elements of a second (independent) category with respective probabilities $p_1$ and $p_2$ result in estimators $\hat{p}_1$ and $\hat{p}_2$ which we declare:

In [ ]:
#Declaration of variables
p_1       = Symbol('p_1')
p_2       = Symbol('p_2')
p_1hat    = RandomSymbol('phat_1')
p_2hat    = RandomSymbol('phat_2')  
N_1       = Symbol('N_1', positive=True, integer = True) #Sample size asssociated to p1
N_2       = Symbol('N_2', positive=True, integer = True) #Sample size asssociated to p2

#Declaration of relative risk function
def RR(p_1, p_2):
  return p_1/p_2

#Declaration of phi for delta method
def phi(p_1, p_2):
  return log(RR(p_1, p_2));

The derivative is given by:

In [ ]:
#Obtain the direction vector
v = Matrix([p_1hat - p_1, p_2hat - p_2])

#And calculate the gradient
grad = derive_by_array(phi(p_1,p_2), [p_1, p_2])
grad = Matrix(grad)

#Thus calculating the Hadamard (directional) derivative:
hadamard = grad.dot(v)
hadamard

-(-p_2 + phat_2)/p_2 + (-p_1 + phat_1)/p_1

And the variance where we use that $\text{Var}(\hat{p}_i) = p_i(1-p_i)/N_i$ and that the covariance is zero due to independence

In [ ]:
variance_expanded = Variance(hadamard).expand().simplify()
variance_expanded

Variance(phat_2)/p_2**2 - 2*Covariance(phat_1, phat_2)/(p_1*p_2) + Variance(phat_1)/p_1**2

Which can be simplified into:

In [ ]:
variance_expanded = variance_expanded.subs(Covariance(p_1hat,p_2hat), 0.0)
variance_expanded = variance_expanded.subs(Variance(p_1hat), p_1*(1 - p_1)/N_1)
variance_expanded = variance_expanded.subs(Variance(p_2hat), p_2*(1 - p_2)/N_2)

variance_expanded

(1 - p_2)/(N_2*p_2) + (1 - p_1)/(N_1*p_1)

## 3.5 Counterexample: When Using the Influence Function Will not Work

Consider the attributable fraction among the exposed for an exposure level $x$ given by:
$$
\textrm{AF}_e(x) = \dfrac{RR(\theta,x)-1}{RR(\theta,x)}.
$$
we compute the expansion around $x = 0$:

In [ ]:
#Declaration of variables
x         = Symbol('x', positive = True)
theta     = Symbol('theta')

#Declaration of relative risk function
def RR(x, theta):
  return exp(x/theta)

#Declaration of phi for delta method
def AF(x, theta):
  return (RR(x,theta) - 1)/RR(x, theta);

#The derivative at x = 0 exists and is zero for positive theta
derivative = diff(AF(x,theta),theta)
limit(derivative, theta, 0)

0

In [ ]:
#This happens for any order (n) of the Taylor expansion as you can verify;
series(AF(1, theta), x0 = 0, n = 10).removeO()

0

## 3.7 Delta-method for the correlation parameter between vectors $X$ and $Y$

We are interested on estimating:

$$
\rho(X,Y)=\dfrac{\mathbb{E}[X Y]−\mathbb{E}[X]\cdot\mathbb{E}[Y]}
{\sqrt{\mathbb{E}[X^{2}]-\mathbb{E}^{2}[X]}\sqrt{\mathbb{E}[Y^{2}]-\mathbb{E}^{2}[Y]}}
$$

where we set $\mu_{X,Y} = \mathbb{E}[X Y]$, $\mu_X = \mathbb{E}[X]$, $\mu_Y = \mathbb{E}[Y]$, $\mu_{X^2} = \mathbb{E}[X^2]$, and $\mu_{Y^2} = \mathbb{E}[Y^2]$. Hence:
$$
\phi(\mu_{X,Y},\mu_X,\mu_Y,\mu_{X^2},\mu_{Y^2}) = \dfrac{\mu_{X,Y} - \mu_X\cdot\mu_Y}{\sqrt{\mu_{X^2} - \mu_X^2}\cdot \sqrt{\mu_{Y^2} - \mu_Y^2}}
$$

In [ ]:
#Create the variables and the estimators
mu_XY    = Symbol('mu_XY')
mu_X     = Symbol('mu_X')
mu_Y     = Symbol('mu_Y')
mu_X2    = Symbol('mu_X2', positive=True)
mu_Y2    = Symbol('mu_Y2', positive=True)
N        = Symbol('N', positive=True, integer=True) #Sample size
XYbar    = RandomSymbol("XYbar")
Xbar     = RandomSymbol("Xbar")
Ybar     = RandomSymbol("Ybar")
Xbar2    = RandomSymbol("X²bar")
Ybar2    = RandomSymbol("Y²bar")

#Declaration of function
def phi(mu_XY, mu_X, mu_Y, mu_X2, mu_Y2):
  return (mu_XY - mu_X*mu_Y)/(sqrt(mu_X2 - pow(mu_X,2))*sqrt(mu_Y2 - pow(mu_Y,2)))

#Obtain the direction vector
v = Matrix([mu_XY - XYbar, mu_X - Xbar, mu_Y - Ybar, mu_X2 - Xbar2, mu_Y2 - Ybar2])

#And calculate the gradient
grad = derive_by_array(phi(mu_XY, mu_X, mu_Y, mu_X2, mu_Y2), [mu_XY, mu_X, mu_Y, mu_X2, mu_Y2])
grad = Matrix(grad)

#Thus calculating the Hadamard (directional) derivative:
hadamard = grad.dot(v)
hadamard

(mu_X - Xbar)*(mu_X*(-mu_X*mu_Y + mu_XY)/((-mu_X**2 + mu_X2)**(3/2)*sqrt(-mu_Y**2 + mu_Y2)) - mu_Y/(sqrt(-mu_X**2 + mu_X2)*sqrt(-mu_Y**2 + mu_Y2))) + (mu_Y - Ybar)*(-mu_X/(sqrt(-mu_X**2 + mu_X2)*sqrt(-mu_Y**2 + mu_Y2)) + mu_Y*(-mu_X*mu_Y + mu_XY)/(sqrt(-mu_X**2 + mu_X2)*(-mu_Y**2 + mu_Y2)**(3/2))) + (mu_XY - XYbar)/(sqrt(-mu_X**2 + mu_X2)*sqrt(-mu_Y**2 + mu_Y2)) - (mu_Y2 - Y²bar)*(-mu_X*mu_Y + mu_XY)/(2*sqrt(-mu_X**2 + mu_X2)*(-mu_Y**2 + mu_Y2)**(3/2)) - (mu_X2 - X²bar)*(-mu_X*mu_Y + mu_XY)/(2*(-mu_X**2 + mu_X2)**(3/2)*sqrt(-mu_Y**2 + mu_Y2))

## 3.8 Applications of the Delta-method in Regression Models 

We are interested in estimating the Relative Risk function given by:
$$
\phi(\beta_0, \beta_1, \beta_2) = \dfrac{1 + e^{-(\beta_0 + \beta_1 + \beta_2)}}{1 + e^{-(\beta_0 + \beta_1)}}.
$$
We can define the function and obtain the gradient:

In [ ]:
#Create the variables and the estimators
beta0      = Symbol('beta_0')
beta1      = Symbol('beta_1')
beta2      = Symbol('beta_2')
betahat_0  = RandomSymbol('betahat_0')
betahat_1  = RandomSymbol('betahat_1')
betahat_2  = RandomSymbol('betahat_2')

#Declaration of function
def phi(beta0, beta1, beta2):
  return (1 + exp(-beta0 - beta1 - beta2))/(1 + exp(-beta0 - beta1))

#Obtain the direction vector
v = Matrix([betahat_0 - beta0, betahat_1 - beta1, betahat_2 - beta2])

#And calculate the gradient
grad = derive_by_array(phi(beta0, beta1, beta2), [beta0, beta1, beta2])
grad = Matrix(grad)


#Thus calculating the Hadamard (directional) derivative:
hadamard = grad.dot(v)
hadamard.simplify()

((beta_2 - betahat_2)*(exp(beta_0 + beta_1) + 1) + (exp(beta_0 + beta_1) - exp(beta_0 + beta_1 + beta_2))*(beta_0 + beta_1 - betahat_0 - betahat_1))*exp(-beta_2)/(exp(beta_0 + beta_1) + 1)**2

In [ ]:
#Against variance covariance matrix
sigma0      = Symbol('sigma^2_0')
sigma1      = Symbol('sigma^2_1')
sigma2      = Symbol('sigma^2_2')
sigma01     = Symbol('sigma_01')
sigma02     = Symbol('sigma_02')
sigma12     = Symbol('sigma_12')

VCOV = Matrix([[sigma0,  sigma01,  sigma02],
               [sigma01,  sigma1,  sigma12],
               [sigma02, sigma12, sigma2]])

varprod = grad.transpose()*VCOV*grad
simplify(simplify(varprod))

Matrix([[((sigma^2_2*(exp(beta_0 + beta_1) + 1) + (sigma_02 + sigma_12)*(exp(beta_0 + beta_1) - exp(beta_0 + beta_1 + beta_2)))*(exp(beta_0 + beta_1) + 1) + (exp(beta_0 + beta_1) - exp(beta_0 + beta_1 + beta_2))*(sigma_02*(exp(beta_0 + beta_1) + 1) + sigma_12*(exp(beta_0 + beta_1) + 1) + (sigma^2_0 + sigma_01)*(exp(beta_0 + beta_1) - exp(beta_0 + beta_1 + beta_2)) + (sigma^2_1 + sigma_01)*(exp(beta_0 + beta_1) - exp(beta_0 + beta_1 + beta_2))))*exp(-2*beta_2)/(exp(beta_0 + beta_1) + 1)**4]])

## Reproducibility

In [ ]:
session_info.show()